# 4. Data Normalization
Let's take a look at few rows from some visits to a doctor in the **`tidy/doctor_visits.csv`** file.

In [1]:
import pandas as pd
import numpy as np
dv = pd.read_csv('../data/tidy/doctor_visits.csv')
dv

,patient_name,visit_date,patient_address,patient_birthdate,clinic_name,clinic_address,cost,doctor_name,doctor_specialty,procedure_name,procedure_code
0,Penelope,5/3/2014,123 Fake Street,4/1/2013,St. Clair,501 Downtown Ave,100,Smith,General Practitioner,Physical,1
1,Abbas,6/1/2014,105 Real Street,3/14/1947,Younge,222 Fortnite Dr,50,Patel,Radiologist,X-ray,20
2,Eleni,6/3/2014,50 Rosegarden,8/9/1981,Main,21 Jayne St,200,Smith,General Practitioner,Physical,1
3,Abbas,6/10/2015,105 Real Street,3/14/1947,Starsky,505 Perry Rd,35,Patel,Radiologist,CT Scan,24
4,Penelope,7/2/2016,123 Fake Street,4/1/2013,Younge,222 Fortnite Dr,44,Brown,Cardiologist,Echocardiogram,31
5,Eleni,3/3/2017,50 Rosegarden,8/9/1981,Younge,222 Fortnite Dr,300,Brown,Cardiologist,Stress Test,36


### The information is easy to read
All the information presented in this table is easy to read. Any question that is asked about the visits can be quickly answered.

### Yet, something is wrong, what is it?
Although the table of data is sufficient to address our questions, it repeats much of the data and will not scale well as more patients come into the system. It will also be difficult to update historical data, such as if the patient's address changes or a clinic changes names.

### A short intro into data normalization
Modern databases attempt to reduce the amount of replication in the data by a process called **normalization**. It involves separating data into tables to minimize replication and increase data accuracy.

From [wikipedia](https://en.wikipedia.org/wiki/Database_normalization):
> Database Normalization, or simply normalization, is the process of organizing the columns (attributes) and tables (relations) of a relational database to reduce data redundancy and improve data integrity. Normalization is also the process of simplifying the design of a database so that it achieves the optimum structure. It reduces and eliminates redundant data. In normalization, data integrity is assured. It was first proposed by Dr. Edgar F. Codd, as an integral part of a relational model.

There is much, much more to data normalization. The following example is just a brief overview.

## Which values are always the same for each visit?
When looking through the table, you should notice that certain values will repeat for every single visit. For instance, the patient name, address, and birth date are going to be the same. There isn't a need to keep repeating all these values for every visit.

### Separating patient values into a distinct table
Let's select all the patient columns in a single DataFrame. The **`copy`** method is called to ensure that this is new DataFrame is a completely different DataFrame and not referring to the same data as the original.

In [2]:
patient = dv[['patient_name', 'patient_address', 'patient_birthdate']]
patient

,patient_name,patient_address,patient_birthdate
0,Penelope,123 Fake Street,4/1/2013
1,Abbas,105 Real Street,3/14/1947
2,Eleni,50 Rosegarden,8/9/1981
3,Abbas,105 Real Street,3/14/1947
4,Penelope,123 Fake Street,4/1/2013
5,Eleni,50 Rosegarden,8/9/1981


### Drop duplicate rows
There is no need to store duplicate values of each row. Let's keep only the unique rows.

In [3]:
patient = patient.drop_duplicates()
patient

,patient_name,patient_address,patient_birthdate
0,Penelope,123 Fake Street,4/1/2013
1,Abbas,105 Real Street,3/14/1947
2,Eleni,50 Rosegarden,8/9/1981


## Create a primary key to uniquely identify each row
When we normalize our data, and separate it into new tables, an additional column is added to the table to uniquely identify each row. The unique value that identifies each row is called a **primary key**. Let's add a primary key to the patient table. Note: If you get the **`SettingWithCopyWarning`**, ignore it.

In [4]:
patient['patient_id'] = np.arange(len(patient))
patient

/Users/Ted/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,patient_name,patient_address,patient_birthdate,patient_id
0,Penelope,123 Fake Street,4/1/2013,0
1,Abbas,105 Real Street,3/14/1947,1
2,Eleni,50 Rosegarden,8/9/1981,2


### Rearrange the columns so that `patient_id` is first
It's best to always put the primary key as the first column.

In [5]:
cols = ['patient_id', 'patient_name', 'patient_address', 'patient_birthdate']
patient = patient[cols]
patient

,patient_id,patient_name,patient_address,patient_birthdate
0,0,Penelope,123 Fake Street,4/1/2013
1,1,Abbas,105 Real Street,3/14/1947
2,2,Eleni,50 Rosegarden,8/9/1981


## We just created a dimension
In database terminology, the patient table would be considered a **dimension**. Dimensions are things that exist in your data that are independent of any event taking place. They tend to be static and do not change often (such as your name or birth date.

## Create tables for all the other dimensions
There are several other dimensions in our original table. Let's create new dimension tables for each one of the following:
* clinic
* doctor
* procedure

We will select each of the columns unique to each dimension, drop the replicated rows and add a primary key as the first column to uniquely identify each row.

### Clinic Dimension

In [6]:
cols = ['clinic_name', 'clinic_address']
clinic = dv[cols].drop_duplicates()
clinic

,clinic_name,clinic_address
0,St. Clair,501 Downtown Ave
1,Younge,222 Fortnite Dr
2,Main,21 Jayne St
3,Starsky,505 Perry Rd


In [7]:
clinic['clinic_id'] = np.arange(len(clinic))
clinic

,clinic_name,clinic_address,clinic_id
0,St. Clair,501 Downtown Ave,0
1,Younge,222 Fortnite Dr,1
2,Main,21 Jayne St,2
3,Starsky,505 Perry Rd,3


In [8]:
cols = ['clinic_id', 'clinic_name', 'clinic_address']
clinic = clinic[cols]
clinic.head()

,clinic_id,clinic_name,clinic_address
0,0,St. Clair,501 Downtown Ave
1,1,Younge,222 Fortnite Dr
2,2,Main,21 Jayne St
3,3,Starsky,505 Perry Rd


### Doctor Dimension

In [9]:
cols = ['doctor_name', 'doctor_specialty']
doctor = dv[cols].drop_duplicates()
doctor

,doctor_name,doctor_specialty
0,Smith,General Practitioner
1,Patel,Radiologist
4,Brown,Cardiologist


In [10]:
doctor['doctor_id'] = np.arange(len(doctor))
doctor

,doctor_name,doctor_specialty,doctor_id
0,Smith,General Practitioner,0
1,Patel,Radiologist,1
4,Brown,Cardiologist,2


In [11]:
cols = ['doctor_id', 'doctor_name', 'doctor_specialty']
doctor = doctor[cols]
doctor.head()

,doctor_id,doctor_name,doctor_specialty
0,0,Smith,General Practitioner
1,1,Patel,Radiologist
4,2,Brown,Cardiologist


### Procedure Dimension
Here, the primary key is already given to us with the procedure code. We will keep it as is.

In [12]:
cols = ['procedure_code', 'procedure_name']
procedure = dv[cols].drop_duplicates()
procedure

,procedure_code,procedure_name
0,1,Physical
1,20,X-ray
3,24,CT Scan
4,31,Echocardiogram
5,36,Stress Test


## Replacing original data with primary keys
We can now revisit our original DataFrame and replace all columns in each dimension with a single column, the primary key of that dimension.

## Join original table to dimension tables
To make the replacement, we will join our original table to each one of our dimension tables. The **`merge`** method joins tables together in Pandas. We can specify how the tables will join with the **`on`** parameter. We will join on all the non-primary key columns. Below, we join the **`patient`** table. The result is one extra column at the end of the DataFrame, the **`patient_id`**.

In [13]:
dv_fact = dv.merge(patient, on=['patient_name', 'patient_address', 'patient_birthdate'])
dv_fact

,patient_name,visit_date,patient_address,patient_birthdate,clinic_name,clinic_address,cost,doctor_name,doctor_specialty,procedure_name,procedure_code,patient_id
0,Penelope,5/3/2014,123 Fake Street,4/1/2013,St. Clair,501 Downtown Ave,100,Smith,General Practitioner,Physical,1,0
1,Penelope,7/2/2016,123 Fake Street,4/1/2013,Younge,222 Fortnite Dr,44,Brown,Cardiologist,Echocardiogram,31,0
2,Abbas,6/1/2014,105 Real Street,3/14/1947,Younge,222 Fortnite Dr,50,Patel,Radiologist,X-ray,20,1
3,Abbas,6/10/2015,105 Real Street,3/14/1947,Starsky,505 Perry Rd,35,Patel,Radiologist,CT Scan,24,1
4,Eleni,6/3/2014,50 Rosegarden,8/9/1981,Main,21 Jayne St,200,Smith,General Practitioner,Physical,1,2
5,Eleni,3/3/2017,50 Rosegarden,8/9/1981,Younge,222 Fortnite Dr,300,Brown,Cardiologist,Stress Test,36,2


### Drop the dimension columns
We can now drop all the original patient columns as the **`patient_id`** now refers to them.

In [14]:
dv_fact = dv_fact.drop(columns=['patient_name', 'patient_address', 'patient_birthdate'])
dv_fact

,visit_date,clinic_name,clinic_address,cost,doctor_name,doctor_specialty,procedure_name,procedure_code,patient_id
0,5/3/2014,St. Clair,501 Downtown Ave,100,Smith,General Practitioner,Physical,1,0
1,7/2/2016,Younge,222 Fortnite Dr,44,Brown,Cardiologist,Echocardiogram,31,0
2,6/1/2014,Younge,222 Fortnite Dr,50,Patel,Radiologist,X-ray,20,1
3,6/10/2015,Starsky,505 Perry Rd,35,Patel,Radiologist,CT Scan,24,1
4,6/3/2014,Main,21 Jayne St,200,Smith,General Practitioner,Physical,1,2
5,3/3/2017,Younge,222 Fortnite Dr,300,Brown,Cardiologist,Stress Test,36,2


## Replace all the other dimensions with primary key columns

### Doctor Dimension

In [15]:
dv_fact = dv_fact.merge(doctor, on=['doctor_name', 'doctor_specialty'])
dv_fact = dv_fact.drop(columns=['doctor_name', 'doctor_specialty'])
dv_fact

,visit_date,clinic_name,clinic_address,cost,procedure_name,procedure_code,patient_id,doctor_id
0,5/3/2014,St. Clair,501 Downtown Ave,100,Physical,1,0,0
1,6/3/2014,Main,21 Jayne St,200,Physical,1,2,0
2,7/2/2016,Younge,222 Fortnite Dr,44,Echocardiogram,31,0,2
3,3/3/2017,Younge,222 Fortnite Dr,300,Stress Test,36,2,2
4,6/1/2014,Younge,222 Fortnite Dr,50,X-ray,20,1,1
5,6/10/2015,Starsky,505 Perry Rd,35,CT Scan,24,1,1


In [16]:
clinic

,clinic_id,clinic_name,clinic_address
0,0,St. Clair,501 Downtown Ave
1,1,Younge,222 Fortnite Dr
2,2,Main,21 Jayne St
3,3,Starsky,505 Perry Rd


### Clinic Dimension

In [17]:
dv_fact = dv_fact.merge(clinic, on=['clinic_name', 'clinic_address'])
dv_fact = dv_fact.drop(columns=['clinic_name', 'clinic_address'])
dv_fact

,visit_date,cost,procedure_name,procedure_code,patient_id,doctor_id,clinic_id
0,5/3/2014,100,Physical,1,0,0,0
1,6/3/2014,200,Physical,1,2,0,2
2,7/2/2016,44,Echocardiogram,31,0,2,1
3,3/3/2017,300,Stress Test,36,2,2,1
4,6/1/2014,50,X-ray,20,1,1,1
5,6/10/2015,35,CT Scan,24,1,1,3


### Procedure Dimension
Since the primary key is already in the table, we can just drop the **`procedure_name`** column

In [18]:
dv_fact = dv_fact.drop(columns=['procedure_name'])
dv_fact

,visit_date,cost,procedure_code,patient_id,doctor_id,clinic_id
0,5/3/2014,100,1,0,0,0
1,6/3/2014,200,1,2,0,2
2,7/2/2016,44,31,0,2,1
3,3/3/2017,300,36,2,2,1
4,6/1/2014,50,20,1,1,1
5,6/10/2015,35,24,1,1,3


### Rearrange columns with foreign keys first
When a primary key from table is found in another table, it is called a **foreign key**. Foreign keys can repeat in the table they are in. Primary keys, however, can never repeat in the tables they are in. This is a very important property. Foreign keys are 

In [19]:
cols = ['patient_id', 'clinic_id', 'doctor_id', 
        'procedure_code', 'visit_date', 'cost']
dv_fact = dv_fact[cols]
dv_fact

,patient_id,clinic_id,doctor_id,procedure_code,visit_date,cost
0,0,0,0,1,5/3/2014,100
1,2,2,0,1,6/3/2014,200
2,0,1,2,31,7/2/2016,44
3,2,1,2,36,3/3/2017,300
4,1,1,1,20,6/1/2014,50
5,1,3,1,24,6/10/2015,35


## Fact Table
This last DataFrame, **`dv_fact`** is called a **fact table** using database terminology. Fact tables hold the actual **events** or **transactions** that take place in a business. They hold the columns that are subject to change such as date and cost here. If we had data from a grocery store, our fact table would have columns like the number of items purchased, the cost of each item, and the type of payment used.  Fact tables have references to the static dimension tables through foreign keys.

# Data Model Diagram
The diagram of the **data model** or **entity-relationship diagram** is presented below. Data models show the logical relationships between the fact and dimension tables. This type of data model is called a **star schema** and one of the simplest designs.

![](images/doctor_data_model.png)

See [this simple Stack Overflow answer][1] for another description of fact and dimension tables.

[1]: https://stackoverflow.com/a/33750545/3707607

# Exercises

### Problem 1
<span  style="color:green; font-size:16px">Tidy the dataset **`tidy/store_transactions.csv`**.</span>